# BJJ 


In [41]:
import urllib.request, urllib.parse, urllib.error
from bs4 import BeautifulSoup
import ssl
import pandas as pd
import numpy as np
import re

In [188]:
# Get all athletes
url = 'https://www.flograppling.com/people'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')
#print(soup)

In [189]:
Athletes = pd.DataFrame(columns=['Athlete_Name','Athlete_ID'], index=range(0,1000))

row = 0
for t in soup.find_all('div', class_ = 'grid-container-item grid-container-item__min-width mb-4 ng-star-inserted'):
    Name = t.get_text()
    Athletes.iloc[row,0] = Name
    for p in t('a'):
        ID = p.get('href')
        Athletes.iloc[row,1] = ID
        row = row + 1
Athletes = Athletes.dropna(axis=0,how='all')
Athletes

,Athlete_Name,Athlete_ID
0,"Aaron ""Tex"" Johnson",/people/6600946-aaron-tex-johnson
1,Adam Wardzinski,/people/5950131-adam-wardzinski
2,Alexssandro Sodre,/people/6749113-alexssandro-sodre
3,Ana Carolina Vieira,/people/5950497-ana-carolina-vieira
4,Andre Galvao,/people/5951184-andre-galvao
...,...,...
97,Vagner Rocha,/people/5950392-vagner-rocha
98,Victor Hugo,/people/6566365-victor-hugo
99,"Vinicius ""Trator"" Ferreira",/people/6708867-vinicius-trator-ferreira
100,Vinny Magalhaes,/people/5950917-vinny-magalhaes


# Table Format
0. Year
1. Date
2. W/L
3. Replay
4. Method
5. Opponent
6. Weight
7. Event

In [150]:
# Sample Athlete
url = 'https://www.flograppling.com/people/6566775-craig-jones'
html = urllib.request.urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')
# Loop
Columns = ['Year','Date','Date_Link','W/L','W/L_Link','Replay','Replay_Link','Method','Method_Link','Opponent','Opponent_Link','Weight','Weight_Link','Event','Event_Link']
Athlete_Matches = pd.DataFrame(columns=Columns,index=range(0,1000)) # Premake table
idx=1
rownum = 0
for t in soup('flo-section-table'): # This will get all the tables on the page
    
    Year = t.find('h5').get_text()
    if Year == 'W/L Methods': # Flo has a table that we don't care about called 'W/L Methods'
        break
    #print(t.find('h6'))
    #print('-------') 
    for td in t.find_all('td'): # td is cell info within a table in html
        link_idx = idx + 1 
        #print(td.get_text())
        text = td.get_text() # the text stored in the cell (what is visible on the website)
        if text == 'Submission': # This line is meant to stop the loop at the end of the tables
            break # Note that Flo has a table with win/loss methods & submission win/loss methods at the end
        link = td('a')[0].get('href') # the link associated with that text (if there is one)

        
        Athlete_Matches.iloc[rownum, idx] = text
        Athlete_Matches.iloc[rownum, link_idx] = link
        #print('Column:',idx,'Year:',Year,'Text:',text,'Link:',link) # Print it all so I can check it works
        idx = idx + 2
        
        if idx == 15: # This is the piece that keeps track of what column it is
            idx=1
            Athlete_Matches.iloc[rownum, 0] = Year # Add year right before we switch the row
            rownum = rownum + 1
    #for piece in t:
        #print(piece.find_all('td'))
Athlete_Matches = Athlete_Matches.dropna(axis=0,how='all') # Get rid of all the extra rows
display(Athlete_Matches)


,Year,Date,Date_Link,W/L,W/L_Link,Replay,Replay_Link,Method,Method_Link,Opponent,Opponent_Link,Weight,Weight_Link,Event,Event_Link
0,2022,09/17,,L,,Replay,/events/7096389/videos?playing=7975476,Points,/events/7096389/videos?playing=7975476,K. Duarte,/people/6451325,-99kgs,,2022 ADCC World Championships,/events/7096389
1,2022,09/17,,W,,Replay,/events/7096389/videos?playing=7975414,Decision,/events/7096389/videos?playing=7975414,N. Meregali,/people/5950129,-99kgs,,2022 ADCC World Championships,/events/7096389
2,2022,09/17,,W,,Replay,/events/7096389/videos?playing=7974649,Guillotine,/events/7096389/videos?playing=7974649,K. Boehm,/people/6600891,-99kgs,,2022 ADCC World Championships,/events/7096389
3,2022,09/17,,W,,Replay,/events/7096389/videos?playing=7974406,Shoulder Lock,/events/7096389/videos?playing=7974406,J. Costa,/people/6808810,-99kgs,,2022 ADCC World Championships,/events/7096389
4,2022,08/27,,W,,,,RNC,,E. Savidov,,Absolute Division,,2022 AIGA Arlan Grip Pro 18,/events/10668133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,2014,03/12,,W,,,,Inside Heel Hook,,T. Spriggs,/people/5950976,---,,2014 Pan Jiu-Jitsu IBJJF Championship,
73,2014,03/12,,D,,,,N/A - Draw,,S. Grecicho,,---,,2014 Pan Jiu-Jitsu IBJJF Championship,
74,2014,03/12,,W,,,,Leg Lock,,M. Verzbickas,,---,,2014 Pan Jiu-Jitsu IBJJF Championship,
75,2014,03/12,,L,,,,Kimura,,R. Barral,/people/5947295,-88kgs,,2014 Pan Jiu-Jitsu IBJJF Championship,
